# Pandas Tricks

*Rappel et/ou découverte de manipulation de matrices de données avec pandas*

### Lire un tableur



In [2]:
import pandas as pd

# préciser le chemin du fichier à lire
cheminDuFichierCSV = "./../data/dictionary/1757_Buchanan-J.csv" 

# demander à pandas de lire le fichier et garder ce tableau accessible dans une variable
dfDicoBuch = pd.read_csv(cheminDuFichierCSV, delimiter=";", encoding="utf8")

# afficher à l'écran un échantillon du tableau
dfDicoBuch.sample(n=5)

,nbSyll,pron,origin,warning,remarques,POS,lineId,pageId
2878,3,cŏrdĕlieʹrs,F.,NaN,NaN,n.,43.0,59
4953,1,fīle,NaN,NaN,NaN,n.,43.0,88
2334,1,cloak,NaN,NaN,NaN,n.,72.0,51
10438,3,pōʹtĕncy,L.,NaN,NaN,n.,53.0,166
1756,1,call,NaN,NaN,NaN,n.,62.0,43


Pour lires des tableurs aux format xls, xlsx ou odt il faut utiliser pd.read_excel(cheminDuFichier) et je conseil de bien préciser explicitement l'engine à utiliser en paramètre.
* dfXlsx = pd.read_excel(cheminDuFichierXLSX, engine="openpyxl")
* dfOds = pd.read_excel(cheminDuFichierODS, engine="odf")

### Ajouter un nouvelle colonne contenant le resultat d'un traitement

In [3]:
# définir une fonction qui retourne quelque chose
def sizePron(strPron):
    #retourner le nombre de caractère de la prononciation
    
    return len(strPron)



In [4]:
def motVedette(mot):
    cheminDuFichierCSV = "./../data/dictionary/Buchanan_PronChar_counts.csv"
    pronchar = pd.read_csv(cheminDuFichierCSV, delimiter=";", encoding="utf8")
    TruePron = []
    transfo = ''
    for lettre in mot:
        if (pronchar[pronchar['char']== lettre]['graph'].isnull().values[0]):
            transfo = ''
        else: transfo = pronchar[pronchar['char']== lettre]['graph'].values[0]
        TruePron.append(transfo)
    TruePron = ''.join(TruePron)
    return TruePron

In [5]:
# appeler cette fonction sur toutes les lignes de la matrice pour remplir une nouvelle colonne grace à apply
dfDicoBuch['sizePron'] = dfDicoBuch.apply(lambda x: sizePron(x['pron']),1)

dfDicoBuch.sample(n=5)

,nbSyll,pron,origin,warning,remarques,POS,lineId,pageId,sizePron
5115,1,flŭx,L.,NaN,NaN,n.,61.0,90,4
3307,4,decĭʹdū_o_us,L.,NaN,NaN,a.,44.0,65,12
10552,4,prepăʹrătive,NaN,NaN,NaN,a.,23.0,168,12
9665,2,păʹmphlet,NaN,NaN,NaN,n.,57.0,155,9
1592,2,brȯʹẇbeat,NaN,NaN,NaN,v.,47.0,41,9


## Un petit calcul

In [6]:
dfDicoBuch['sizePron'].median()

9.0

## Une recherche de valeur

In [7]:
dfDicoBuch[dfDicoBuch['sizePron']==26.0]

,nbSyll,pron,origin,warning,remarques,POS,lineId,pageId,sizePron
2813,3,contrŏʹller or comptrollĕr,NaN,NaN,NaN,n.,49.0,58,26
7492,3,kaʹlĕnd_a_r or caʹlĕnd_a_r,NaN,NaN,NaN,n.,2.0,125,26


## Afficher des graphiques
Plusieurs librairie possibles :
* matplotlib (static, bonne qualité pour publication)
* seaborn (interactif, pour le web)
* bokeh (interactif, pour le web)
* plotly (interactif, pour le web)

In [8]:
import seaborn as sns
import matplotlib.pyplot as plt

In [9]:
%matplotlib ipympl

plt.figure("scatter plot view")
sns.scatterplot(data=dfDicoBuch, x="nbSyll", y="sizePron", hue="POS")

ModuleNotFoundError: No module named 'ipympl'

In [10]:
%matplotlib ipympl

plt.figure("scatter plot view")
sns.scatterplot(data=dfDicoBuch, x="nbSyll", y="sizePron", hue="POS", alpha=0.05)

ModuleNotFoundError: No module named 'ipympl'

## Manipuler des matrices de format exotique

In [11]:
cheminDuDicoBigi = "./../data/dictionary/2015_Bigi-B.dict" 

# ouvrir une matric exotique
#dfDicoBigi = pd.read_csv(cheminDuDicoBigi, delimiter=" [] ", encoding="utf8")

dfDicoBigi = pd.read_csv(cheminDuDicoBigi, delimiter=" \[\] ", encoding="utf8", engine='python', names=['hwd','pronSampa'], skiprows=8 )

# afficher à l'écran un échantillon du tableau
dfDicoBigi.head(n=5)

,hwd,pronSampa
0,a,@
1,a(2),eI
2,a'ready,@ r\ E d i:
3,a's,eI z
4,a.,eI


Intro/rappel Regex : https://gitlab.huma-num.fr/mnauge/cnam-athon-transcrire/-/blob/master/codes/regexBasics.ipynb

## Jouer avec deux matrices à la fois

In [12]:
cheminDuMappingSampaIpa = "./../data/dictionary/sampa-ipa.txt" 
dfMappingSampaIpa = pd.read_csv(cheminDuMappingSampaIpa, delimiter="  *", encoding="utf8", engine='python', names=['sampa','ipa'])
dfMappingSampaIpa.sample(n=3)

,sampa,ipa
174,z\,ʑ
114,_t,̤
153,q,q


In [13]:
dfMappingSampaIpa['sampa'].value_counts()

r       3
h       2
@       2
U~/     2
g       2
       ..
X       1
J       1
N       1
_B_L    1
U       1
Name: sampa, Length: 177, dtype: int64

In [14]:
dfMappingSampaIpa[dfMappingSampaIpa['sampa']=='@']

,sampa,ipa
28,@,ə
29,@,ɛ


In [15]:
# ne garder que la première occurence pour éviter une indécision mais ce n'est pas terrible. A discuter avec les linguistes.
dfMappingSampaIpa.drop_duplicates(subset=['sampa'], keep='first', inplace=True)

In [16]:
dfMappingSampaIpa[dfMappingSampaIpa['sampa']=='@']

,sampa,ipa
28,@,ə


In [17]:
def sampa2ipa(strSampa, dfMapping):
    ipaRes = ""
    
    listValSampa = strSampa.split(" ")
    #print(listValSampa)
    for valSampa in listValSampa:
        dfRowFinded = dfMapping[dfMapping['sampa']==valSampa]

        if len(dfRowFinded==1):
            ipaRes+=dfRowFinded.iloc[0]['ipa']
        else:
            ipaRes+='?'
    return ipaRes

In [18]:
sampa2ipa("A r\ t i",dfMappingSampaIpa)

'ɑɹti'

In [19]:
dfDicoBigiRich = dfDicoBigi.copy()

dfDicoBigiRich['ipa'] = dfDicoBigiRich.apply(lambda x: sampa2ipa(x['pronSampa'], dfMappingSampaIpa),1)


### Sauvegarder la nouvelle matrice


In [20]:
pathDicoBigiRich = "./../data/dataframe/BigiRichSample.csv"
dfDicoBigiRich.to_csv(pathDicoBigiRich, sep=";",encoding="utf8",index=False)